<a href="https://colab.research.google.com/github/Palaeoprot/pFind/blob/main/FASTA_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To parse FASTA files
Attempts to understand different headers on FASTA files and read this into a dataframe
Then using the 'Genus' to fill in a taxonomic heirarchy
It may make errors!

## The FASTA file format

As we have seen, sequence data is ubiquitous in computational biology and bioinformatics. One of the most straightforward methods of representing that data is as a FASTA file. [FASTA format](https://en.wikipedia.org/wiki/FASTA_format) files are simply text files. They may have various file extensions including .txt, .fa, .fasta, .fna or .faa. Typically, .fna indicates a nucleotide fasta file, whereas .faa indicates an amino acid fasta file. The name FASTA format orginates from the name of an old program, FASTA, that was the first to use this format. But now many different programs in bioinformatics use this format.

## The structure of FASTA files

Here's how FASTA files are structured:

- FASTA files can contain one or more sequences
- Each sequence entry has two main parts, an **identifier line** and one or more **sequence lines**
- The **identifier line** *must* begin with a `>` (greater than) character. This line says what the sequence is. The identifier line also indicates that all following lines are sequence, until you reach either another identifier line or the end of the file. In some cases, identifier lines *may* also contain additional information about the sequence - like it's putative function - but this entirely depends on the specific database or research group that wrote the FASTA file.
The identifier line is: `>tr|R9WNI0|R9WNI0_HUMAN Fragile X mental retardation 1 OS=Homo sapiens OX=9606 GN=FMR1 PE=1 SV=1`

- **Sequence line(s).** the line(s) of a FASTA file after an identifier line, contain the actual sequence data. In some cases, long sequences will simply be placed on a single very long line. In other FASTA files, like this one, long sequences are broken up into multiple lines, each with no more than a certain number of characters (typically 60).

In computer science, parsers ) (see e.g. [parsing](https://en.wikipedia.org/wiki/Parsing)) are pieces of code that take  is the process of taking some input and interpreting it in terms of data structures that are useful within your program.

Building on our ability to open and print the contents of a FASTA file in python, let's build a FASTA parser step by step. Once we're done, we should have all our sequences properly associated with their names in python, and we should be able to look up each sequence easily by its identifier.



In [2]:
from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setting up

In [ ]:
import re
import pandas as pd
import json
from collections import OrderedDict

## Global Variables

In [ ]:
#STUDY_NAME = 'CatePinkWhtPaste'
STUDY_NAME = 'Dinosaur'

BASE_PATH = f'/content/drive/MyDrive/Colab_Notebooks/NovorCloud/{STUDY_NAME}'
# DENOVO_PATH = f'{BASE_PATH}/{SAMPLE_NAME}.denovo.csv'
FOLDER_PREFIX = 'F_2 - '
SAMPLE_NAME = '260423_Turkey_Collagen_1hr_1_in_100'
#SAMPLE_NAME = '20230614-0314_QEHF3_1005195_ONJ_TR_MatthewEvo_14_LLM_C_PinkP_12_ICib'
PSMS_PATH = f'{BASE_PATH}/{FOLDER_PREFIX}{SAMPLE_NAME}/{SAMPLE_NAME}.psm.csv'
# PEPS_PATH = f'{BASE_PATH}/{SAMPLE_NAME}.peps.txt'
# FASTA_PATH = f'{BASE_PATH}/{SAMPLE_NAME}.proteins.fasta'
#COMBINED_FASTA_PATH = f'{BASE_PATH}/{STUDY_NAME}_deduplicated.proteins.fasta'

gene_list = ["COL1A1", "COL1A2", "COL3A1"]
gene_name = 'COL1A1'

#Functions

In [1]:
#NovorClouud FASTA
def parse_novor_fasta(fasta_file):
    with open(fasta_file, 'r') as file:
        fasta_data = file.read()

    records = fasta_data.strip().split('>')
    parsed_data = []

    for record in records:
        if not record:
            continue

        lines = record.split('\n')
        header = lines[0]
        sequence = ''.join(lines[1:])

        # Extract gene
        gene = re.search(r'COL1A1\w*', header, re.IGNORECASE)
        gene = gene.group() if gene else 'Unknown'

        # Sequence length (excluding specified characters)
        seq_length = len(re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', sequence))

        # Extract genus and species
        match = re.search(r'\b[A-Z][a-z]*\b \b[a-z]+\b', header)
        if match:
            genus, species = match.group().split()
        else:
            genus, species = 'Unknown', 'Unknown'

        parsed_data.append([gene, header, sequence, seq_length, genus, species])

    return parsed_data

#NovorCloud FASTA parsing

def parse_novor_fasta(fasta_file):
    with open(fasta_file, 'r') as file:
        fasta_data = file.read()

    records = fasta_data.strip().split('>')
    parsed_data = []

    for record in records:
        if not record:
            continue

        lines = record.split('\n')
        header = lines[0]
        sequence = ''.join(lines[1:])

        # Extract gene
        gene = re.search(r'COL1A1\w*', header, re.IGNORECASE)
        gene = gene.group() if gene else 'Unknown'

        # Sequence length (excluding specified characters)
        seq_length = len(re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', sequence))

        # Extract genus and species
        match = re.search(r'\b[A-Z][a-z]*\b \b[a-z]+\b', header)
        if match:
            genus, species = match.group().split()
        else:
            genus, species = 'Unknown', 'Unknown'

        parsed_data.append([gene, header, sequence, seq_length, genus, species])

    return parsed_data

def clean_gene_name(gene_name):
    """
    Simplify the gene name by removing unwanted characters and possibly
    normalizing the format if necessary.

    Parameters:
    gene_name (str): The full gene name to clean.

    Returns:
    str: A cleaned-up version of the gene name.
    """
    # Example of simplification, adapt as needed
    return re.sub(r'[^a-zA-Z0-9\s]', '', gene_name).strip()

def parse_fasta_header(header):
    """
    Parse the FASTA header to extract sequence ID, cleaned gene name,
    species name, and GeneID.

    Parameters:
    header (str): The FASTA header string.

    Returns:
    tuple: A tuple containing sequence ID, cleaned gene name, species name, and GeneID.
    """
    parts = header.split('|')
    seq_id = parts[0].strip()
    gene_name = clean_gene_name(parts[1].strip())
    species_name = re.search(r'\[(.*?)\]', header).group(1)
    gene_id = re.search(r'#(\d+)#', header).group(1)

    return seq_id, gene_name, species_name, gene_id

def process_fasta_files(base_path, study_name):
    """
    Process all FASTA files within the given base path, combining and
    deduplicating records based on GeneID.

    Parameters:
    base_path (str): The base directory containing FASTA files.
    study_name (str): The name of the study for output file naming.

    Returns:
    None
    """
    combined_records = OrderedDict()

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.fasta'):
                fasta_path = os.path.join(root, file)
                with open(fasta_path, 'r') as fasta_file:
                    records = fasta_file.read().strip().split('>')
                    for record in records:
                        if not record:
                            continue
                        lines = record.split('\n')
                        header = lines[0]
                        sequence = ''.join(lines[1:])
                        seq_id, gene_name, species_name, gene_id = parse_fasta_header(header)
                        combined_records[gene_id] = (seq_id, gene_name, species_name, sequence)

    combined_fasta_path = f'{base_path}/{study_name}_deduplicated.proteins.fasta'
    with open(combined_fasta_path, 'w') as output_file:
        for gene_id, (seq_id, gene_name, species_name, sequence) in combined_records.items():
            output_file.write(f'>{seq_id} {gene_name} [{species_name}] | |#{gene_id}#\n{sequence}\n')


#More traditional fasta files
def parse_NCBI_fasta(input_file):
    """Return a dict of {id:protein_seq} pairs based on the sequences in the input FASTA file
    input_file -- a file handle for an input fasta file
    """
    parsed_seqs = {}
    curr_seq_id = None
    curr_seq = []

    for line in f:
        line = line.strip()

        if line.startswith(">"):
            if curr_seq_id is not None:
                parsed_seqs[curr_seq_id] = ''.join(curr_seq)

            curr_seq_id = line[1:]
            curr_seq = []
            continue

        curr_seq.append(line)

    #Add the final sequence to the dict
    parsed_seqs[curr_seq_id] = ''.join(curr_seq)
    return parsed_seqs

import re

def parse_messy_fasta(file_name, gene):
    """
    Parses a FASTA file with potentially messy headers to extract and clean specific data.

    This function reads a FASTA file, cleans up each sequence's header to extract relevant biological information,
    and filters sequences by a specified gene name. The output is a list of cleaned records, each containing
    the gene name, the original header, the sequence, its length, and the extracted biological taxonomy
    information (genus and species).

    Parameters:
    - file_name: str. The path to the FASTA file.
    - gene: str. The gene name to filter sequences by.

    Returns:
    - parsed_records: list of str. Each element is a comma-separated string containing:
                      the gene name, original header, cleaned sequence, sequence length,
                      gene name extracted from the header (if any), genus, and species.
    """

    records = []
    current_header = None
    current_seq = ''

    with open(file_name) as f:
        for line in f:
            line = line.strip()

            if line.startswith('>'):
                if current_header:
                    # Append the record before starting a new one
                    records.append((gene, current_header, current_seq, len(current_seq)))
                    current_seq = ''
                # Remove '>' and start new header
                current_header = line[1:]
            else:
                # Clean and append sequence lines, removing any characters not A-Y
                current_seq += re.sub('[^A-Y]', '', line)

    # Append the last record
    if current_header:
        records.append((gene, current_header, current_seq, len(current_seq)))

    parsed_records = []
    for gene, header, sequence, seq_length in records:
        # Attempt to extract genus and species from the header
        name_parts = re.search(r'OS=([^ O]+) ([^ O]+)', header)
        if not name_parts:
            name_parts = re.search(r'\|[^|]+\|\s*([^ ]*)\s*([^ ]*)', header)

        genus = species = str_gene = ''
        if name_parts:
            genus, species = name_parts.groups()

        # Extract gene name if present in the header
        str_gene_match = re.search(r'col\w*', header, re.IGNORECASE)
        if str_gene_match:
            str_gene = str_gene_match.group(0)

        # Prepare and append the cleaned record
        parsed_records.append(','.join([gene, header, sequence, str(seq_length), str_gene, genus, species]))

    return parsed_records

In [ ]:
# EProcess fasta files

process_fasta_files(BASE_PATH, STUDY_NAME)



In [ ]:
# def parse_fasta (fasta_file):
#     with open(fasta_file, 'r') as file:
#         fasta_data = file.read()

#     records = fasta_data.strip().split('>')
#     parsed_data = []

#     for record in records:
#         if not record:
#             continue

#         lines = record.split('\n')
#         header = lines[0]
#         sequence = ''.join(lines[1:])

#         # Extract gene
#         gene = re.search(r'COL1A1\w*', header, re.IGNORECASE)
#         gene = gene.group() if gene else 'Unknown'

#         # Sequence length (excluding specified characters)
#         seq_length = len(re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', sequence))

#         # Extract genus and species
#         match = re.search(r'\b[A-Z][a-z]*\b \b[a-z]+\b', header)
#         if match:
#             genus, species = match.group().split()
#         else:
#             genus, species = 'Unknown', 'Unknown'

#         parsed_data.append([gene, header, sequence, seq_length, genus, species])

#     return parsed_data

In [ ]:
# Function: load_dict
# Loads a master dictionary from a file.
# This function reads a JSON file and returns its contents as a dictionary.
#
# Parameters:
# file_path (str): The file path to the JSON file containing the dictionary.
#
# Returns:
# dict: A dictionary representing the contents of the dictionary file.
# ------------------------------------------------------------------------------
def load_dict(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)


In [ ]:
def process_record(header, sequence):
    """Processes a single record and extracts the desired information.

    Args:
        header: The header string.
        sequence: The amino acid sequence.

    Returns:
        A dictionary containing the extracted information.
    """

    record = {
        "gene": "COL1A1",  # Always set to COL1A1
        "header": header,
        "sequence": sequence,
        "seq_length": count_aa_length(sequence),
    }

    # Extract genus and species from header
    genus, species = extract_genus_species(header)
    record["genus"] = genus
    record["species"] = species

    # Extract gene name from header (if found)
    str_gene = extract_gene_name(header)
    record["str_gene"] = str_gene

    return record

In [ ]:
def count_aa_length(sequence):
    """Counts the length of the amino acid sequence, excluding non-AA characters."""
    return len(re.sub(r"[^ACDEFGHIKLMNPQRSTVWY]", "", sequence))


In [ ]:
def extract_genus_species(header):
    """Extracts genus and species names from the header using multiple patterns."""
    patterns = [
        r"^(\w+)\_(\w+)$",  # Pattern 1: genus_species
        r"OS=(\w+)\s(\w+)",  # Pattern 2: OS=genus species
        r"\|(\w+)\|(\w+)\|",  # Pattern 3: |genus|species|
    ]
    for pattern in patterns:
        match = re.search(pattern, header)
        if match:
            return match.groups()

    # If no pattern matches, use a placeholder
    return "Unknown", "Unknown"

In [ ]:
def extract_gene_name(header):
    """Extracts the gene name from the header (if found)."""
    gene_name_pattern = r"COL1A1|col1a1b|col1b"
    match = re.search(gene_name_pattern, header, flags=re.IGNORECASE)
    return match.group(0) if match else None

 Reading and writing FASTA files

This section describes how to read and write biological sequences stored in FASTA files.

#### In this section you will learn
* How to read and write text files in python
* How sequence data are represented in the FASTA file format
* How to download data from an online address using `urlretrieve`
* How to check if a file is in our current directory using `listdir`
* How to work with special tab (`'\t'`) and newline(`'\n'`) characters
* How to combine `if` statements and `for` loops to read a FASTA file in python
* How to use the `continue` keyword to skip one iteration of a `for` loop in python

#### Prerequisites for this section
* Have Anaconda python installed
* Be familiar with how to run python
* Be familiar with the Central Dogma of Molecular Biology, and how to represent DNA,RNA, and Protein sequences using python strings.
* Use `for` loops to simplify repetitive code
* Use `if` statements to let your code handle different conditions




# Parsing FASTA
Here is a simple function for parsing a FASTA file into a Python dictionary. The dictionary maps short names to corresponding nucleotide strings (with whitespace removed).

In [ ]:
def parse_fasta(fh):
    fa = {}
    current_short_name = None
    # Part 1: compile list of lines per sequence
    for ln in fh:
        if ln[0] == '>':
            # new name line; remember current sequence's short name
            long_name = ln[1:].rstrip()
            current_short_name = long_name.split()[0]
            fa[current_short_name] = []
        else:
            # append nucleotides to current sequence
            fa[current_short_name].append(ln.rstrip())
    # Part 2: join lists into strings
    for short_name, nuc_list in fa.items():
        # join this sequence's lines into one long string
        fa[short_name] = ''.join(nuc_list)
    return fa

The first part accumulates a list of strings (one per line) for each sequence. The second part joins those lines together so that we end up with one long string per sequence. Why divide it up this way? Mainly to avoid the poor performance of repeatedly concatenating (immutable) Python strings.

I'll test it by running it on the simple multi-FASTA file we saw before:

In [ ]:
from io import StringIO
fasta_example = StringIO(
'''>sequence1_short_name with optional additional info after whitespace
ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAA
GCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGC
AATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT
>sequence2_short_name with optional additional info after whitespace
GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCG
ATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATA
ATATAG''')
parsed_fa = parse_fasta(fasta_example)
parsed_fa

Accessing a substring in this way is very fast and simple. The downside is that you've stored all of the sequences in memory. If the FASTA files are really big, this takes lots of valuable memory. This may or may not be a good trade.

An alternative is to load only the portions of the FASTA files that you need, when you need them. For this to be practical, we have to have a way of "jumping" to the specific part of the specific FASTA file that you're intersted in.

Fortunately, there is a standard way of indexing a FASTA file, popularized by the faidx tool in SAMtools. When you have such an index, it's easy to calculate exactly where to jump to when you want to extract a specific substring. Here is some Python to create such an index

In [ ]:
def index_fasta(fh):
    index = []
    current_short_name = None
    current_byte_offset, running_seq_length, running_byte_offset = 0, 0, 0
    line_length_including_ws, line_length_excluding_ws = 0, 0
    for ln in fh:
        ln_stripped = ln.rstrip()
        running_byte_offset += len(ln)
        if ln[0] == '>':
            if current_short_name is not None:
                index.append((current_short_name, running_seq_length,
                              current_byte_offset, line_length_excluding_ws,
                              line_length_including_ws))
            long_name = ln_stripped[1:]
            current_short_name = long_name.split()[0]
            current_byte_offset = running_byte_offset
            running_seq_length = 0
        else:
            line_length_including_ws = max(line_length_including_ws, len(ln))
            line_length_excluding_ws = max(line_length_excluding_ws, len(ln_stripped))
            running_seq_length += len(ln_stripped)
    if current_short_name is not None:
        index.append((current_short_name, running_seq_length,
                      current_byte_offset, line_length_excluding_ws,
                      line_length_including_ws))
    return index

What do the fields in those two records mean?  Take the first record: `('sequence1_short_name', 194, 69, 70, 71)`.  The fields from left to right are (1) the short name, (2) the length (in nucleotides), (3) the byte offset in the FASTA file of the first nucleotide of the sequence, (4) the maximum number of nucleotides per line, and (5) the maximum number of *bytes* per line, including whitespace.  It's not hard to convince yourself that, if you know all these things, it's not hard to figure out the byte offset of any position in any of the sequences.  (This is what the `get` member of the `FastaIndexed` class defined below does.)

A typical way to build a FASTA index like this is to use [SAMtools], specifically the `samtools faidx` command.  This and all the other `samtools` commands are documented in [its manual](http://samtools.sourceforge.net/samtools.shtml).

[SAMtools]: http://samtools.sourceforge.net

When you use a tool like this to index a FASTA file, a new file containing the index is written with an additional `.fai` extension.  E.g. if the FASTA file is named `hg19.fa`, then running `samtools faidx hg19.fa` will create a new file `hg19.fa.fai` containing the index.

The following Python class shows how you might use the FASTA file together with its index to extract arbitrary substrings without loading all of the sequences into memory:

In [ ]:
import re

class FastaOOB(Exception):
    """ Out-of-bounds exception for FASTA sequences """

    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)

class FastaIndexed(object):
    """ Encapsulates a set of indexed FASTA files.  Does not load the FASTA
        files into memory but still allows the user to extract arbitrary
        substrings, with the help of the index. """

    __removeWs = re.compile(r'\s+')

    def __init__(self, fafns):
        self.fafhs = {}
        self.faidxs = {}
        self.chr2fh = {}
        self.offset = {}
        self.lens = {}
        self.charsPerLine = {}
        self.bytesPerLine = {}

        for fafn in fafns:
            # Open FASTA file
            self.fafhs[fafn] = fh = open(fafn, 'r')
            # Parse corresponding .fai file
            with open(fafn + '.fai') as idxfh:
                for ln in idxfh:
                    toks = ln.rstrip().split()
                    if len(toks) == 0:
                        continue
                    assert len(toks) == 5
                    # Parse and save the index line
                    chr, ln, offset, charsPerLine, bytesPerLine = toks
                    self.chr2fh[chr] = fh
                    self.offset[chr] = int(offset) # 0-based
                    self.lens[chr] = int(ln)
                    self.charsPerLine[chr] = int(charsPerLine)
                    self.bytesPerLine[chr] = int(bytesPerLine)

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        # Close all the open FASTA files
        for fafh in self.fafhs.values():
            fafh.close()

    def has_name(self, refid):
        return refid in self.offset

    def name_iter(self):
        return self.offset.iterkeys()

    def length_of_ref(self, refid):
        return self.lens[refid]

    def get(self, refid, start, ln):
        ''' Return the specified substring of the reference. '''
        assert refid in self.offset
        if start + ln > self.lens[refid]:
            raise ReferenceOOB('"%s" has length %d; tried to get [%d, %d)' % (refid, self.lens[refid], start, start + ln))
        fh, offset, charsPerLine, bytesPerLine = \
            self.chr2fh[refid], self.offset[refid], \
            self.charsPerLine[refid], self.bytesPerLine[refid]
        byteOff = offset
        byteOff += (start // charsPerLine) * bytesPerLine
        into = start % charsPerLine
        byteOff += into
        fh.seek(byteOff)
        left = charsPerLine - into
        # Count the number of line breaks interrupting the rest of the
        # string we're trying to read
        if ln < left:
            return fh.read(ln)
        else:
            nbreaks = 1 + (ln - left) // charsPerLine
            res = fh.read(ln + nbreaks * (bytesPerLine - charsPerLine))
            res = re.sub(self.__removeWs, '', res)
        return res


# Main Program

In [ ]:
# Fastafile path
#fasta_file_path = f'/content/drive/MyDrive/Colab_Notebooks/Collagen Sequence/RegExRactorFASTAin/{gene_name}.fasta'
fasta_file_path = ('/content/drive/MyDrive/Colab_Notebooks/NovorCloud/Tuuli_Masks/Tuuli_Mask_DA288.proteins.fasta')
pparsed_fasta_data = parse_fasta(fasta_file_path)

# Creating a DataFrame
df = pd.DataFrame(parsed_fasta_data, columns=['gene', 'header', 'sequence', 'length', 'genus', 'species'])

genus_dict = load_dict('/content/drive/MyDrive/Colab_Notebooks/Collagen Sequence/TaxonID/Genus_Unified_Dictionary.json')

# Fill in other taxonomic levels using the genus dictionary
taxonomic_levels = ['phylum', 'class', 'superorder', 'order', 'family']
for level in taxonomic_levels:
    df[level] = df['genus'].apply(lambda x: genus_dict.get(x, {}).get(level))



# # For example, to print the data:
# for data in parsed_fasta_data:
#     print(data)



# Display the DataFrame
print(df)


NameError: name 'parsed_fasta_data' is not defined

In [ ]:
# records = []
# current_header = None
# current_seq = ''

# with open('sequences.fa') as f:
#     for line in f:
#         line = line.strip()

#         if line.startswith('>'):
#             if current_header:
#                 records.append((gene, current_header, current_seq, len(current_seq)))
#                 current_seq = ''

#             current_header = line[1:]
#         else:
#             current_seq += re.sub('[^A-Y]', '', line)

# if current_header:
#     records.append((gene, current_header, current_seq, len(current_seq)))

# for record in records:
#     header = record[1]

#     name_parts = re.search(r'OS=([^ O]+) ([^ O]+)', header)
#     if not name_parts:
#         name_parts = re.search(r'\|[^|]+\|\s*([^ ]*)\s*([^ ]*)', header)

#     genus = species = str_gene = ''
#     if name_parts:
#         genus, species = name_parts.groups()

#     str_gene = re.search(r'col\w*', header, re.IGNORECASE)
#     if str_gene:
#         str_gene = str_gene.group(0)

#     print(','.join([record[0], header, record[2], str(record[3]), str_gene, genus, species]))

In [ ]:


parsed_fasta_data = parse_fasta(fasta_file_path)

# Output the parsed data or convert it to a CSV or DataFrame for further use.
